## Fake News Detection: Data Science Project

### Importing the packages

In [ ]:
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model

In [ ]:
# importing libraries and packages
import numpy as np
import pandas as pd
import json
import csv
import random

#importing the tensorflow to preprocessing the data and to create the model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()



In [ ]:
data = pd.read_csv("/content/drive/MyDrive/data/news.csv")
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


### Data Preprocessing

In [ ]:
data = data.drop(["Unnamed: 0"], axis = 1)
data.head(5)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
# data encoding
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

In [ ]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_size = 3000
test_portion = .1

In [ ]:
#Tokenization
title = []
text = []
labels = []

for x in range(training_size):
  title.append(data['title'][x])
  text.append(data['text'][x])
  labels.append(data['label'][x])

In [ ]:
#applying the tokenization
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
    sequences1, padding = padding_type, truncating = trunc_type
)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

In [ ]:
#generating word embedding
embeddings_index = {}
with open('/content/drive/MyDrive/data/glove.6B.50d.txt') as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    embeddings_index[word] = coefs

embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim))
for word, i in word_index1.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embeddings_matrix[i] = embedding_vector

### Creating model arcitecture

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size1+1, embedding_dim,
                              input_length = max_length, weights = [
                                  embeddings_matrix],
                              trainable = False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(pool_size = 4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer = 'adam', metrics = ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 54, 50)            377600    
                                                                 
 dropout_1 (Dropout)         (None, 54, 50)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 64)            16064     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 12, 64)            0         
 g1D)                                                            
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                      

In [ ]:
from keras.src.engine import training
num_epochs = 50

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels,
                    epochs = num_epochs,
                    validation_data = (testing_padded, testing_labels),
                    verbose = 2)

Train on 2700 samples, validate on 300 samples
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2700/2700 - 2s - loss: 0.6450 - accuracy: 0.6200 - val_loss: 0.5863 - val_accuracy: 0.6667 - 2s/epoch - 811us/sample
Epoch 2/50
2700/2700 - 1s - loss: 0.5897 - accuracy: 0.6859 - val_loss: 0.5410 - val_accuracy: 0.6800 - 1s/epoch - 552us/sample
Epoch 3/50
2700/2700 - 1s - loss: 0.5387 - accuracy: 0.7307 - val_loss: 0.5264 - val_accuracy: 0.7033 - 1s/epoch - 547us/sample
Epoch 4/50
2700/2700 - 1s - loss: 0.5003 - accuracy: 0.7600 - val_loss: 0.5194 - val_accuracy: 0.7267 - 1s/epoch - 537us/sample
Epoch 5/50
2700/2700 - 1s - loss: 0.4518 - accuracy: 0.7926 - val_loss: 0.5177 - val_accuracy: 0.7100 - 1s/epoch - 541us/sample
Epoch 6/50
2700/2700 - 2s - loss: 0.4084 - accuracy: 0.8159 - val_loss: 0.5636 - val_accuracy: 0.7167 - 2s/epoch - 693us/sample
Epoch 7/50
2700/2700 - 3s - loss: 0.3709 - accuracy: 0.8348 - val_loss: 0.5127 - val_accuracy: 0.7167 - 3s/epoch - 1ms/sample
Epoch 8/50
2700/2700 - 3s - loss: 0.3304 - accuracy: 0.8585 - val_loss: 0.5170 - val_accuracy: 0.7467 - 3s/epoch - 97

### Model Evaluation and Prediction

In [ ]:
#Providing the sample text and validating

X = "Suyash is going to be the next big thing"

#detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen = 54,
                          padding = padding_type,
                          truncating = trunc_type)

if(model.predict(sequences, verbose = 0)[0][0] >= 0.5):
  print("This news is REAL")

else:
  print("This news is FAKE")

This news is REAL
